In [1]:
using Printf, CSV, DataFrames, Statistics, Glob, XLSX, Dates, Plots, TimeSeries, JSON

In [2]:
directory = joinpath(homedir(), "research", "GADS_Processed", "raw_data")
files = glob("gen_*_type_200_dtgrp_*_rating_*_state_Texas_raw.csv", directory)

4-element Vector{String}:
 "/home/emco4286/research/GADS_Pr" ⋯ 40 bytes ⋯ "_3_rating_3_state_Texas_raw.csv"
 "/home/emco4286/research/GADS_Pr" ⋯ 40 bytes ⋯ "_3_rating_3_state_Texas_raw.csv"
 "/home/emco4286/research/GADS_Pr" ⋯ 40 bytes ⋯ "_2_rating_3_state_Texas_raw.csv"
 "/home/emco4286/research/GADS_Pr" ⋯ 40 bytes ⋯ "_2_rating_3_state_Texas_raw.csv"

In [ ]:
# dtgrp = 2 -> South Texas Project, 1280 MW, online ~1989, COAST LOAD ZONE
# dtgrp = 3 -> Commanche Peak, 1210 MW, online ~1993, NORTH CENTRAL LOAD ZONE

In [3]:
x = Dict()
i = 1
mydict = Dict([("A", 1), ("RS", 1), ("NC", 1),  ("IR", 1), ("MB", 1), ("F", 2), ("U1", 2), ("U2" , 2), ("U3" , 2), ("D1", 2), ("D2", 2), ("D3", 2), ("SF", 2), ("PO", 3), ("PE", 3), ("PD", 3), ("DP", 3), ("MO", 4), ("ME", 4), ("DM", 4), ("D4", 4), ("RU", 5), ("DR", 5), ("TC", 5)])
# agedict = Dict([(1, DateTime(1980, 1, 1)), (2, DateTime(1985, 1, 1)), (3, DateTime(1995, 1, 1)), (4, DateTime(2005, 1, 1)), (5, DateTime(2015, 1, 1)), (6, DateTime(2020, 1, 1))])
agedict = Dict([(2, DateTime(1989, 1, 1)), (3, DateTime(1993, 1, 1))])
types_dict = Dict("EventStartDT" => DateTime, "EventEndDT" => DateTime)
rating_dict = Dict([(1, 50), (2, 150), (3, 200)])
weather_zone_dict = Dict([(2, "COAST"), (3, "NORTH_C")]);

In [4]:
fname = "ercot_hourly_load_2013_2023.csv"
path = joinpath(homedir(), "research", "ERCOT_LOAD_DATA", "processed", fname)
load_data = DataFrame(CSV.File(path))
DataFrames.rename!(load_data, :Hour_End => "x")

Row,x,COAST,NORTH_C
,DateTime,Float64,Float64
1,2013-01-01T01:00:00,7607.19,10370.3
2,2013-01-01T02:00:00,7389.27,10153.9
3,2013-01-01T03:00:00,7179.75,9989.24
4,2013-01-01T04:00:00,7039.69,9947.86
5,2013-01-01T05:00:00,6991.41,10097.4
6,2013-01-01T06:00:00,7050.73,10472.1
7,2013-01-01T07:00:00,7177.79,10891.2
8,2013-01-01T08:00:00,7223.21,11190.2
9,2013-01-01T09:00:00,7314.25,11627.1


In [5]:
start_date_g = DateTime(2013,1,1,6)
end_date_g = DateTime(2024,1,1,6)
date_range_g = [k for k in start_date_g:Hour(1):end_date_g]

myrange_g = [k for k in 0:1:round(end_date_g - start_date_g, Hour).value]

for f in files

    mymatch = match(r"/home/emco4286/research/GADS_Processed/raw_data/gen_(\d+)_type_(\d+)_dtgrp_(\d+)_rating_(\d+)_state_Texas_raw.csv", f)
    id = parse(Int32, mymatch[1])
    type = parse(Int32, mymatch[2])
    dtgrp = parse(Int32, mymatch[3])
    rating = parse(Int32, mymatch[4])

    df = DataFrame(CSV.File(f, types=types_dict))
    df = df[typeof.(df.EventEndDT) .== DateTime, :] # for missing DateTime values
    df = df[df.EventTypeCode .!= "RS", :]
    sort!(df, :EventStartDT)
    transform!(df, :EventTypeCode => ByRow(x -> mydict[x]), :EventTypeCode)

    # Generator-specific characteristics

    load = weather_zone_dict[df[1, :CommercialDT_grp]]
    birthdate = agedict[df[1, :CommercialDT_grp]]

    # 
    state = convert(Array{Union{Missing, Float64}}, fill(1, length(myrange_g))) # to allow missing values
    cause = convert(Array{Union{Missing, Float64}}, fill(1, length(myrange_g)))
    age = [k.value for k in round.(date_range_g - birthdate, Day(1))]


    # Hour that outage starts
    time_vector = map(x -> round(x - start_date_g, Hour), df[!, :EventStartDT])

    # Set state to zero during an event
    for k in [c for c in range(1, step=1, length=length(time_vector))] # k is an index
        start_time_string = round(df[k, :EventStartDT], Hour)
        end_time_string = round(df[k, :EventEndDT], Hour)

        mycause = df[k, :EventTypeCode_function]

        # if cause == 1 # RS
        #     continue
        # end

        start_hour = time_vector[k].value # integer
        duration = round(end_time_string - start_time_string, Hour).value
        end_hour = start_hour + duration

        start_index = max(1, start_hour)

        matching_idx = findall(x -> x == end_hour, myrange_g)
        
        if length(matching_idx) < 1
            end_index = length(state)
        else
            end_index = matching_idx[1]
        end

        # If event is unit getting retired, set state as missing until the end of the period and skip to the next unit
        if cause == 5
            state[start_index:end] .= missing
        else
            state[start_index:end_index] .= 0
            cause[start_index:end_index] .= mycause
        end

    end

    trajectory = DataFrame(x=date_range_g, y1=state, y2=cause, y3=age)
    merged = innerjoin(trajectory, load_data[!,["x", load]], on=:x)
    sort!(merged, :x)
    merged[!, :y4] = Dates.dayofyear.(merged[!, :x])

    fname = @sprintf "gen_%d_type_%d_dtgrp_%d_rating_%d_state_Texas_raw.csv" id type dtgrp rating 
    path = joinpath(homedir(), "research", "GADS_Processed", "trajectories_with_features", fname)
    CSV.write(path, merged)

end